In [2]:
%load_ext autoreload
%autoreload 2

In [131]:
import plotly.graph_objects as go

In [4]:
import scopone_scientifico as ss

In [135]:
tournament = ss.Tournament(
    team1=('A', 'B'), 
    team2=('C', 'D'),
    n_match=1
)

tournament.run()
tournament.matches[0].hands[0].show_logs()

In [130]:
# fig = go.Figure(
#     data=[go.Histogram(x=tournament.scoreboard['hands'])]
# )
# fig.show()